In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

device = "mps" if torch.backends.mps.is_available() else "cpu"
dtype = torch.float16 if device == "mps" else torch.float32

model_name = "Qwen/Qwen3-1.7B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=dtype,
)
model.to(device)
model.eval()

/Users/Patron/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/Patron/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.30s/it]


Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 2048)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=2048, out_features=6144, bias=False)
          (up_proj): Linear(in_features=2048, out_features=6144, bias=False)
          (down_proj): Linear(in_features=6144, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): Qwe

In [6]:
prompt = "Give me the reason for anemia."
messages = [{"role": "user", "content": prompt}]

# Disable thinking mode
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False  # important: disable hidden reasoning
)

model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# Generate answer
with torch.inference_mode():
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=512,
        pad_token_id=tokenizer.eos_token_id
    )

# Decode only the answer
answer = tokenizer.decode(
    generated_ids[0][model_inputs.input_ids.shape[1]:],
    skip_special_tokens=True
).strip()

print(answer)

Anemia is a medical condition in which the blood does not carry enough oxygen to the body's tissues. This can happen for a variety of reasons, and the causes can be classified into several categories:

---

### **1. Causes of Anemia**

#### **(a) Blood Loss**
- **Gastrointestinal (GI) bleeding**: Bleeding in the stomach, intestines, or rectum (e.g., from ulcers, diverticulitis, or hemorrhoids).
- **Urogenital bleeding**: Bleeding from the bladder, urethra, or vagina (e.g., from infections, tumors, or menopause).
- **Trauma or surgery**: Blood loss during or after surgery.
- **Menstruation**: Heavy menstrual bleeding in women.

#### **(b) Reduced Production of Red Blood Cells (RBCs)**
- **Iron deficiency**: The most common cause of anemia. Iron is essential for hemoglobin production.
  - **Iron deficiency**: Due to poor absorption, dietary deficiency, or blood loss.
  - **Hemoglobin synthesis**: Lack of iron, vitamin B12, or folate.
- **Aplastic anemia**: Bone marrow failure that preven

In [7]:
save_dir = "./qwen3-1.7B"

tokenizer.save_pretrained(save_dir)
model.save_pretrained(save_dir)